In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 1   # 两个设施点
m = 1   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [3]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
# Hp = generate_Hp(n, m, d, g)

from qiskit import transpile
# 输入qc, 返回电路酉矩阵
def get_circ_unitary(quantum_circuit):
  backend = Aer.get_backend('unitary_simulator' )
  new_circuit = transpile(quantum_circuit, backend)
  job = backend.run(new_circuit)
  result = job.result()
  unitary = result.get_unitary()
  return unitary

In [4]:
import sys
sys.path.append('../')
import z_library.linear_system as ls

In [5]:
def gnrt_gate_hdi(u):
    result = []
    nonzero_indices = ls.find_nonzero_indices(u)
    for urow, nrow in zip(u, nonzero_indices):
        # 把非0元素映射成01
        filtered_arr = [0 if x == -1 else 1 for x in urow if x != 0]
        binary_str = ''.join(map(str, filtered_arr))
        # 取到二进制表示的数
        map_num = int(binary_str, 2)
        # print(map_num)
        length = len(nrow)
        scale = 2**length
        matrix = np.zeros((scale, scale))

        matrix[map_num, scale - 1 - map_num] = 1
        map_num = 2**length - 1 - map_num
        # print(map_num)
        matrix[map_num, scale - 1 - map_num] = 1
        result.append(matrix)
    return result

# print("非零元素的索引：", nonzero_indices)

u = np.array([[-1,  1, 0], [0, -1, 1]])
print(u)
nonzero_indices = ls.find_nonzero_indices(u)
gate_hds = gnrt_gate_hdi(u)
print(nonzero_indices)

[[-1  1  0]
 [ 0 -1  1]]
[[0, 1], [1, 2]]


In [6]:
# gnrt_gate_hdi(u)[0]

In [7]:
from functools import reduce

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))
    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) @ np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) @ np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])
        hamiltonian += term1 + term2
    return hamiltonian

import sys
sys.path.append('../')
import z_library.linear_system as ls
# u = ls.gnrt_u(n, m)

v = np.where(u == 1, 1, 0)
w = np.where(u == -1, 1, 0)
Hd = calculate_hamiltonian(v, w)

In [8]:
print(Hd.shape)
print(gate_hds[0].shape)
np.array_equal(Hd, gate_hds[0])

(8, 8)
(4, 4)


False

In [9]:
from scipy.linalg import expm
def build_circ(params):
  qc = QuantumCircuit(num_qubits)
  gamma = params
  for gate_hdi, ind in zip(gate_hds, nonzero_indices):
    qc.unitary(expm(-1j * gamma * gate_hdi), (num_qubits - 1 - i for i in ind))
  return qc

In [10]:
from scipy.linalg import expm
def build_circ2(params):
  qc = QuantumCircuit(num_qubits)
  gamma = params
  # for dp in range(depth):
  #   # qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
  qc.unitary(expm(-1j * gamma * Hd), range(num_qubits))
  # qc.measure_all()
  return qc

In [11]:
from scipy.linalg import expm
def build_circ3(params):
  qc = QuantumCircuit(num_qubits)
  gamma = params
  # for dp in range(depth):
  #   # qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
  qc.unitary(expm(-1j * Hd), range(num_qubits-1, -1, -1))
  # qc.measure_all()
  return qc

def build_circ4(params):
  qc = QuantumCircuit(num_qubits)
  gamma = params
  # for dp in range(depth):
  #   # qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
  qc.unitary(expm(-1j * Hd), range(num_qubits))
  # qc.measure_all()
  return qc

qc3 = build_circ3(1)
qc4 = build_circ4(1)
m3 = get_circ_unitary(qc3) 
m4 = get_circ_unitary(qc4) 
np.where(m3 != m4)

(array([], dtype=int64),)

In [12]:
Hd

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [13]:
from qiskit import transpile
ls.set_print_form()


qc1 = build_circ(1)
qc2 = build_circ2(1)
m2 = get_circ_unitary(qc1) 
m1 = get_circ_unitary(qc2) 
np.where(m1 != m2)

(array([0]),)

In [14]:
qc1.draw()

┌──────────┐
q_0: ────────────┤1         ├
     ┌──────────┐│  Unitary │
q_1: ┤1         ├┤0         ├
     │  Unitary │└──────────┘
q_2: ┤0         ├────────────
     └──────────┘

In [15]:
qc2.draw()

┌──────────┐
q_0: ┤0         ├
     │          │
q_1: ┤1 Unitary ├
     │          │
q_2: ┤2         ├
     └──────────┘